In [ ]:
import numpy as np
import pandas as pd
import os
import glob
import matplotlib.pyplot as plt
import statistics
import seaborn as sns
from pathlib import Path
import re
import requests
from io import StringIO
import random
import pdb

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Upload trips data
trips = pd.read_csv('/content/drive/MyDrive/Code/Results/trips2.csv',index_col=0)
trips

,Prov_Ori,Prov_Des,Com_Ori,Com_Des,Mun_Ori,Mun_Des,Act_Ori,Act_Des,Proposito,Modo,...,Motos,Tipo_familia,Sexo,Edad,Actividad,crnt_tur,crnt_mot,crnt_otr,Año,Income
0,20,20,Donostialdea,Donostialdea,Donostia/San Sebastian,Donostia/San Sebastian,Otra vivienda,Centro estudios,Otros,Autobús urbano,...,1,2 adultos con niño(s),2,2,4,2,2,2,2021,0.00
1,20,20,Donostialdea,Donostialdea,Donostia/San Sebastian,Donostia/San Sebastian,Centro estudios,Compras doméstico-familiares,Estudio,Andando,...,1,2 adultos con niño(s),2,2,4,2,2,2,2021,0.00
2,20,20,Donostialdea,Donostialdea,Donostia/San Sebastian,Donostia/San Sebastian,Compras doméstico-familiares,Otra vivienda,Compras,Andando,...,1,2 adultos con niño(s),2,2,4,2,2,2,2021,0.00
3,20,20,Donostialdea,Goierri,Donostia/San Sebastian,Beasain,Asuntos trabajo,Asuntos trabajo,Trabajo,Coche conductor,...,1,2 adultos con niño(s),1,5,1,1,2,1,2021,25397.86
4,20,20,Debagoiena,Donostialdea,Arrasate/Mondragon,Donostia/San Sebastian,Asuntos trabajo,Asuntos trabajo,Trabajo,Coche conductor,...,1,2 adultos con niño(s),1,5,1,1,2,2,2021,27611.51
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21899,20,20,Donostialdea,Donostialdea,Hernani,Hernani,Gestiones personales,Residencia Habitual,Gestiones Personales,Coche conductor,...,1,Hogar de una persona,1,6,3,1,2,2,2011,23841.61
21900,20,20,Goierri,Goierri,Zumarraga,Zumarraga,Residencia Habitual,Trabajo habitual,Trabajo,Coche conductor,...,1,2 adultos,1,5,1,1,2,2,2011,24108.52
21901,20,20,Goierri,Goierri,Zumarraga,Zumarraga,Trabajo habitual,Residencia Habitual,Trabajo,Coche conductor,...,1,2 adultos,1,5,1,1,2,2,2011,25703.71
21902,20,20,Goierri,Goierri,Zumarraga,Zumarraga,Residencia Habitual,Compras personales,Compras,Andando,...,1,2 adultos,2,5,6,2,2,2,2011,11982.27


In [ ]:
# Quedarse con commute de trabajo
trips = trips[(trips['Act_Ori'].isin(['Trabajo habitual', 'Asuntos trabajo'])) | (trips['Act_Des'].isin(['Trabajo habitual', 'Asuntos trabajo']))]
trips = trips.reset_index(drop=True)

# 2021 poblacion actual
trips = trips[trips['Año'] == 2021]
trips

,Prov_Ori,Prov_Des,Com_Ori,Com_Des,Mun_Ori,Mun_Des,Act_Ori,Act_Des,Proposito,Modo,...,Motos,Tipo_familia,Sexo,Edad,Actividad,crnt_tur,crnt_mot,crnt_otr,Año,Income
0,20,20,Donostialdea,Goierri,Donostia/San Sebastian,Beasain,Asuntos trabajo,Asuntos trabajo,Trabajo,Coche conductor,...,1,2 adultos con niño(s),1,5,1,1,2,1,2021,25397.860
1,20,20,Debagoiena,Donostialdea,Arrasate/Mondragon,Donostia/San Sebastian,Asuntos trabajo,Asuntos trabajo,Trabajo,Coche conductor,...,1,2 adultos con niño(s),1,5,1,1,2,2,2021,27611.510
2,20,20,Donostialdea,Donostialdea,Urnieta,Urnieta,Trabajo habitual,"Ocio, cultura, deporte",Trabajo,Coche conductor,...,1,2 adultos,1,5,1,1,1,1,2021,26229.910
3,20,20,Donostialdea,Donostialdea,Urnieta,Urnieta,"Ocio, cultura, deporte",Trabajo habitual,Ocio,Coche conductor,...,1,2 adultos,1,5,1,1,1,1,2021,26229.910
4,20,20,Donostialdea,Urola Kosta,Andoain,Zarautz,Trabajo habitual,Gestiones personales,Trabajo,Coche conductor,...,1,2 adultos con niño(s),1,5,1,1,2,2,2021,25397.860
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2205,20,20,Debagoiena,Goierri,Bergara,Zumarraga,Trabajo habitual,Residencia Habitual,Trabajo,Coche conductor,...,1,Otros hogares con niños,2,5,1,1,2,2,2021,23335.260
2206,20,20,Goierri,Goierri,Zumarraga,Zumarraga,Trabajo habitual,Residencia Habitual,Trabajo,Andando,...,1,Otros hogares con niños,1,3,1,1,2,2,2021,23819.585
2207,20,20,Goierri,Goierri,Zumarraga,Zumarraga,Residencia Habitual,Trabajo habitual,Trabajo,Andando,...,1,Otros hogares con niños,1,3,1,1,2,2,2021,21452.435
2208,20,20,Donostialdea,Donostialdea,Lasarte-Oria,Donostia/San Sebastian,Residencia Habitual,Asuntos trabajo,Trabajo,Coche conductor,...,1,Hogar de una persona,1,5,1,1,2,2,2021,26198.010


In [ ]:
trips['elev'] = trips['elev']/trips['elev'].min()
trips['elev'] = trips['elev'].apply(lambda x: int(x) if isinstance(x, float) else x)

In [ ]:
# Expandir los trips, hacer que cada trayecto sea una fila

# Crear una lista de filas duplicadas
nuevas_filas = []
for index, row in trips.iterrows():
    valores_columnas = row[trips.columns]
    repeticiones = row['elev']
    nuevas_filas.extend([valores_columnas] * repeticiones)

# Crear un nuevo DataFrame con las filas duplicadas
trips = pd.DataFrame(nuevas_filas, columns=trips.columns)

# Elimino la columna desplazamientos que ya no tiene sentido alguno
trips = trips.drop(columns=["elev"])

# Mezclo las filas de manera aleatoria
trips = trips.sample(frac=1).reset_index(drop=True)
trips

,Prov_Ori,Prov_Des,Com_Ori,Com_Des,Mun_Ori,Mun_Des,Act_Ori,Act_Des,Proposito,Modo,...,Motos,Tipo_familia,Sexo,Edad,Actividad,crnt_tur,crnt_mot,crnt_otr,Año,Income
0,20,20,Donostialdea,Donostialdea,Donostia/San Sebastian,Donostia/San Sebastian,Trabajo habitual,"Acompañamiento de ayuda, cuidado",Trabajo,Coche conductor,...,2,Otros hogares sin niños,2,4,1,1,2,2,2021,26401.96
1,20,20,Donostialdea,Donostialdea,Donostia/San Sebastian,Donostia/San Sebastian,Trabajo habitual,Residencia Habitual,Trabajo,Coche conductor,...,1,2 adultos,2,5,1,1,2,2,2021,27299.26
2,20,20,Urola Kosta,Urola Kosta,Orio,Zarautz,Residencia Habitual,Trabajo habitual,Trabajo,Coche conductor,...,2,2 adultos,2,5,1,1,2,2,2021,24662.51
3,20,20,Bidasoa,Urola Kosta,Irun,Zarautz,Residencia Habitual,Trabajo habitual,Trabajo,Coche conductor,...,1,Hogar de una persona,1,4,1,1,2,2,2021,23263.22
4,20,20,Donostialdea,Donostialdea,Donostia/San Sebastian,Donostia/San Sebastian,Residencia Habitual,Trabajo habitual,Trabajo,Coche conductor,...,1,2 adultos con niño(s),1,5,1,1,2,2,2021,28185.41
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10711,20,20,Donostialdea,Donostialdea,Donostia/San Sebastian,Oiartzun,Residencia Habitual,Trabajo habitual,Trabajo,Coche conductor,...,2,Otros hogares sin niños,2,5,1,1,2,2,2021,26892.91
10712,20,20,Donostialdea,Bidasoa,Donostia/San Sebastian,Hondarribia,Trabajo habitual,Residencia Habitual,Trabajo,Coche pasajero,...,1,Otros hogares sin niños,1,4,1,2,2,2,2021,24992.16
10713,20,20,Donostialdea,Donostialdea,Donostia/San Sebastian,Donostia/San Sebastian,"Acompañamiento de ayuda, cuidado",Trabajo habitual,Gestiones Personales,Coche conductor,...,2,2 adultos con niño(s),1,4,1,1,1,2,2021,26958.71
10714,20,20,Goierri,Goierri,Urretxu,Urretxu,Residencia Habitual,Trabajo habitual,Trabajo,Andando,...,1,2 adultos con niño(s),1,5,1,1,2,2,2021,25500.46


In [ ]:
trips = trips[trips['Mun_Ori'] != trips['Mun_Des']]
# trips = trips[(trips['Com_Ori'] == 'Donostialdea') | (trips['Com_Des'] == 'Donostialdea')]
trips = trips[trips['Com_Des'] == 'Donostialdea']
trips = trips[trips['Income'] > 1] # Quito a los estudiantes que hacen trips de trabajo
trips

,Prov_Ori,Prov_Des,Com_Ori,Com_Des,Mun_Ori,Mun_Des,Act_Ori,Act_Des,Proposito,Modo,...,Motos,Tipo_familia,Sexo,Edad,Actividad,crnt_tur,crnt_mot,crnt_otr,Año,Income
19,20,20,Donostialdea,Donostialdea,Oiartzun,Errenteria,Residencia Habitual,Trabajo habitual,Trabajo,Coche conductor,...,2,Otros hogares con niños,2,3,1,1,2,2,2021,23763.435
23,20,20,Urola Kosta,Donostialdea,Orio,Andoain,Residencia Habitual,Trabajo habitual,Trabajo,Coche conductor,...,1,Hogar de una persona,1,4,1,1,2,2,2021,25309.110
26,20,20,Donostialdea,Donostialdea,Donostia/San Sebastian,Hernani,Trabajo habitual,Residencia Habitual,Trabajo,Coche conductor,...,1,Hogar de una persona,2,5,1,1,2,2,2021,22911.920
46,20,20,Debagoiena,Donostialdea,Bergara,Usurbil,Trabajo habitual,Residencia Habitual,Trabajo,Coche conductor,...,1,Otros hogares sin niños,2,4,1,1,2,2,2021,25997.260
50,20,20,Donostialdea,Donostialdea,Donostia/San Sebastian,Urnieta,Trabajo habitual,Residencia Habitual,Trabajo,Coche conductor,...,1,Otros hogares sin niños,1,5,1,1,2,2,2021,28048.410
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10701,20,20,Bidasoa,Donostialdea,Irun,Oiartzun,Trabajo habitual,Residencia Habitual,Trabajo,Coche conductor,...,1,2 adultos con niño(s),1,5,1,1,1,2,2021,25461.260
10704,20,20,Goierri,Donostialdea,Legazpi,Donostia/San Sebastian,Residencia Habitual,Asuntos trabajo,Trabajo,Coche conductor,...,1,Hogar de una persona,1,5,1,1,2,2,2021,23463.100
10705,20,20,Bidasoa,Donostialdea,Irun,Donostia/San Sebastian,Residencia Habitual,Trabajo habitual,Trabajo,Coche conductor,...,1,2 adultos con niño(s),1,6,1,1,2,2,2021,24718.410
10708,20,20,Donostialdea,Donostialdea,Donostia/San Sebastian,Pasaia,Residencia Habitual,Trabajo habitual,Trabajo,Andando,...,1,2 adultos con niño(s),2,5,1,1,2,2,2021,22504.870


# Asigno LAT Y LONG Origen y Destino

In [ ]:
import geopandas as gpd
# Leer el archivo Shapefile
gdf = gpd.read_file('/content/drive/MyDrive/Code/Data/ERAIKINAK_EDIFICIOS/buildings.shp')

# Pasar las geometrias de los edificios de sistema UTM a sistema decimal de coordenadas
crs_objetivo = 'EPSG:4326'  # WGS84
gdf = gdf.to_crs(crs_objetivo)

# Crear una columna que contiene el centroide del edificio, para obtener un punto
gdf['Centroide'] = gdf['geometry'].centroid
gdf.columns

<ipython-input-8-42cfb7b7801c>:10: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  gdf['Centroide'] = gdf['geometry'].centroid


Index(['Town', 'Edificabil', 'Solar', 'Vivienda', 'Trastero', 'Garaje Cer',
       'Agropecuar', 'Local Come', 'Industrial', 'Almacen', 'Deportivo',
       'Cultural', 'Educativo', 'Oficina', 'Hosteleria', 'Obras Urba',
       'Altura men', 'Terraza', 'Accesos y', 'Jardin', 'Sanidad', 'Deposito',
       'Duplex', 'Plaza Park', 'Religioso', 'Espectacul', 'geometry',
       'Centroide'],
      dtype='object')

In [ ]:
asignaciones = {
    'Otra vivienda': ['Vivienda', 'Duplex'],
    'Centro estudios': ['Educativo'],
    'Compras doméstico-familiares': ['Local Come'],
    'Asuntos trabajo': ['Oficina'],
    'Trabajo habitual': ['Oficina'],
    'Ocio, cultura, deporte': ['Deportivo', 'Cultural', 'Hosteleria', 'Terraza', 'Jardin', 'Religioso', 'Espectacul'],
    'Acompañamiento de ayuda, cuidado': ['Local Come'],
    'Residencia Habitual': ['Vivienda', 'Duplex'],
    'Médico / Hospital': ['Sanidad'],
    'Gestiones personales': ['Local Come'],
    'Compras personales': ['Local Come'],
    'Otras actividades': ['Local Come'],
    'Gestiones doméstico-familiares': ['Local Come'],

}

# Encontrar la longitud máxima de todas las listas
max_length = max(len(lista) for lista in asignaciones.values())

# Rellenar las listas con valores NaN para que tengan la misma longitud
for columna in asignaciones:
    asignaciones[columna] += [np.nan] * (max_length - len(asignaciones[columna]))

# Crear el DataFrame a partir del diccionario
asignaciones = pd.DataFrame(asignaciones)

In [ ]:
# Función para obtener la lista de una columna específica
def obtener_lista_por_columna(df, columna):
    if columna in df.columns:
        l = df[columna].tolist()
        l = [x for x in l if isinstance(x, str)]
        return l
    else:
        return []

# Función para asignar coord_origen basado en la probabilidad de vivienda
def asignar_coord_origen(row):
    matching_rows = gdf[gdf['Town'] == row['Mun_Ori']] # Lista con todos las RC de ese pueblo
    matching_usos = obtener_lista_por_columna(asignaciones, row['Act_Ori']) # Lista con los posibles usos de esa actividad
    matching_rows = matching_rows[(matching_rows[matching_usos] != 0).any(axis=1)] # De las RC del pueblo, coge solo las que no tienen nulo el m2, mínimo alguno de los usos de suelo asignado a esa actividad origen. Esto es, si hay alguna RC que no tiene ningún uso de suelo de interés para esa actividad, se elimina.

    # Genero una matriz auxiliar quitando Town, geometry y Centroide para hacer los calculos
    auxiliar = matching_rows.drop(columns=['Town', 'geometry', 'Centroide'])
    # Hasta este punto, tengo un gdf con las RC de pueblo adecuado y con todas las RC que hacen match con esa actividad, esto es, no hay ninguna fila que tenga todos los m2 nulos.
    # El próximo paso es asignar una fila (RC), y para ello escojo por probabilidad una ubicación exacta de la matriz (tanto RC como uso de suelo) y selecciono esa fila (RC) para pasarle

    if not matching_rows.empty:
        probabilidades = auxiliar.values / auxiliar.values.sum() # Normalizar los valores para representar probabilidades
        # Generar números aleatorios para seleccionar fila y columna
        fila_seleccionada = random.choices(range(auxiliar.shape[0]), weights=probabilidades.sum(axis=1))[0]
        columna_seleccionada = random.choices(range(auxiliar.shape[1]), weights=probabilidades[fila_seleccionada])[0]
        valor_seleccionado = auxiliar.iloc[fila_seleccionada, columna_seleccionada]         # Obtener el valor seleccionado, fila y columna
        fila = matching_rows.index[fila_seleccionada] # Devolver la fila, la columna y el valor seleccionado
        return gdf.loc[fila, 'Centroide']
    else:
        return None

trips['Edif_Ori'] = trips.apply(asignar_coord_origen, axis=1)


In [ ]:
# Pasa de un punto en formato WKT a dos columnas con LONG y LAT

from shapely import wkt
from shapely.geometry import Point

def extraer_longitud(punto):
    return punto.x

def extraer_latitud(punto):
    return punto.y

# from shapely import wkt
# from shapely.geometry import Point

# def extraer_longitud(punto_wkt):
#     punto = wkt.loads(punto_wkt)
#     return punto.x

# def extraer_latitud(punto_wkt):
#     punto = wkt.loads(punto_wkt)
#     return punto.y

trips['O_long'] = trips['Edif_Ori'].apply(extraer_longitud)
trips['O_lat'] = trips['Edif_Ori'].apply(extraer_latitud)


# Eliminar columnas innecesarias

trips = trips.drop(columns=["Edif_Ori"])
trips = trips.reset_index(drop=True)
trips

,Prov_Ori,Prov_Des,Com_Ori,Com_Des,Mun_Ori,Mun_Des,Act_Ori,Act_Des,Proposito,Modo,...,Sexo,Edad,Actividad,crnt_tur,crnt_mot,crnt_otr,Año,Income,O_long,O_lat
0,20,20,Donostialdea,Donostialdea,Oiartzun,Errenteria,Residencia Habitual,Trabajo habitual,Trabajo,Coche conductor,...,2,3,1,1,2,2,2021,23763.435,-1.869476,43.299272
1,20,20,Urola Kosta,Donostialdea,Orio,Andoain,Residencia Habitual,Trabajo habitual,Trabajo,Coche conductor,...,1,4,1,1,2,2,2021,25309.110,-2.128351,43.276954
2,20,20,Donostialdea,Donostialdea,Donostia/San Sebastian,Hernani,Trabajo habitual,Residencia Habitual,Trabajo,Coche conductor,...,2,5,1,1,2,2,2021,22911.920,-1.977797,43.303195
3,20,20,Debagoiena,Donostialdea,Bergara,Usurbil,Trabajo habitual,Residencia Habitual,Trabajo,Coche conductor,...,2,4,1,1,2,2,2021,25997.260,-2.423608,43.104149
4,20,20,Donostialdea,Donostialdea,Donostia/San Sebastian,Urnieta,Trabajo habitual,Residencia Habitual,Trabajo,Coche conductor,...,1,5,1,1,2,2,2021,28048.410,-2.019292,43.306946
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2457,20,20,Bidasoa,Donostialdea,Irun,Oiartzun,Trabajo habitual,Residencia Habitual,Trabajo,Coche conductor,...,1,5,1,1,1,2,2021,25461.260,-1.761189,43.339585
2458,20,20,Goierri,Donostialdea,Legazpi,Donostia/San Sebastian,Residencia Habitual,Asuntos trabajo,Trabajo,Coche conductor,...,1,5,1,1,2,2,2021,23463.100,-2.341281,43.059045
2459,20,20,Bidasoa,Donostialdea,Irun,Donostia/San Sebastian,Residencia Habitual,Trabajo habitual,Trabajo,Coche conductor,...,1,6,1,1,2,2,2021,24718.410,-1.798813,43.343189
2460,20,20,Donostialdea,Donostialdea,Donostia/San Sebastian,Pasaia,Residencia Habitual,Trabajo habitual,Trabajo,Andando,...,2,5,1,1,2,2,2021,22504.870,-2.003906,43.311991


In [ ]:
trips['D_long'] = -2.033693
trips['D_lat'] = 43.256347

In [ ]:
trips.columns

Index(['Prov_Ori', 'Prov_Des', 'Com_Ori', 'Com_Des', 'Mun_Ori', 'Mun_Des',
       'Act_Ori', 'Act_Des', 'Proposito', 'Modo', 'Hora_Ini', 'Dur_Tot',
       'Recur', 'Bill', 'Park', 'Peaje', 'Provincia', 'Municipio', 'Comarca',
       'Per_hog', 'Turismos', 'Motos', 'Tipo_familia', 'Sexo', 'Edad',
       'Actividad', 'crnt_tur', 'crnt_mot', 'crnt_otr', 'Año', 'Income',
       'O_long', 'O_lat', 'D_long', 'D_lat'],
      dtype='object')

In [ ]:
# Me quedo con las filas que sirven para entrenar el modelo

eliminar = ['Prov_Ori', 'Prov_Des',
       'Act_Ori', 'Act_Des', 'Proposito', 'Dur_Tot',
       'Bill', 'Park', 'Peaje', 'Provincia','Comarca',
       'crnt_mot', 'crnt_otr']
trips = trips.drop(columns=eliminar)
trips

,Com_Ori,Com_Des,Mun_Ori,Mun_Des,Modo,Hora_Ini,Recur,Municipio,Per_hog,Turismos,...,Sexo,Edad,Actividad,crnt_tur,Año,Income,O_long,O_lat,D_long,D_lat
0,Donostialdea,Donostialdea,Oiartzun,Errenteria,Coche conductor,14:50,22,Oiartzun,2,3,...,2,3,1,1,2021,23763.435,-1.869476,43.299272,-2.033693,43.256347
1,Urola Kosta,Donostialdea,Orio,Andoain,Coche conductor,20:40,22,Orio,1,2,...,1,4,1,1,2021,25309.110,-2.128351,43.276954,-2.033693,43.256347
2,Donostialdea,Donostialdea,Donostia/San Sebastian,Hernani,Coche conductor,20:00,22,Hernani,1,2,...,2,5,1,1,2021,22911.920,-1.977797,43.303195,-2.033693,43.256347
3,Debagoiena,Donostialdea,Bergara,Usurbil,Coche conductor,15:00,22,Usurbil,2,4,...,2,4,1,1,2021,25997.260,-2.423608,43.104149,-2.033693,43.256347
4,Donostialdea,Donostialdea,Donostia/San Sebastian,Urnieta,Coche conductor,14:00,23,Urnieta,2,2,...,1,5,1,1,2021,28048.410,-2.019292,43.306946,-2.033693,43.256347
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2457,Bidasoa,Donostialdea,Irun,Oiartzun,Coche conductor,13:00,22,Oiartzun,2,3,...,1,5,1,1,2021,25461.260,-1.761189,43.339585,-2.033693,43.256347
2458,Goierri,Donostialdea,Legazpi,Donostia/San Sebastian,Coche conductor,08:00,99,Legazpi,1,2,...,1,5,1,1,2021,23463.100,-2.341281,43.059045,-2.033693,43.256347
2459,Bidasoa,Donostialdea,Irun,Donostia/San Sebastian,Coche conductor,05:05,22,Irun,2,2,...,1,6,1,1,2021,24718.410,-1.798813,43.343189,-2.033693,43.256347
2460,Donostialdea,Donostialdea,Donostia/San Sebastian,Pasaia,Andando,20:30,23,Donostia/San Sebastian,2,2,...,2,5,1,1,2021,22504.870,-2.003906,43.311991,-2.033693,43.256347


In [ ]:
trips.to_csv('/content/drive/MyDrive/Code/Results/workers_eskuzaitzeta.csv')